In [ ]:
# Copyright (C) 2025 NVIDIA Corporation.  All rights reserved.
#
# This work is licensed under the LICENSE file
# located at the root directory.
import torch
import random

from visualization_utils import show_images
from addit_flux_pipeline import AdditFluxPipeline
from addit_flux_transformer import AdditFluxTransformer2DModel
from addit_scheduler import AdditFlowMatchEulerDiscreteScheduler
from addit_methods import add_object_generated

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
my_transformer  = AdditFluxTransformer2DModel.from_pretrained("black-forest-labs/FLUX.1-dev", subfolder="transformer", torch_dtype=torch.bfloat16)

pipe = AdditFluxPipeline.from_pretrained("black-forest-labs/FLUX.1-dev", 
                                      transformer=my_transformer,
                                      torch_dtype=torch.bfloat16).to(device)
pipe.scheduler = AdditFlowMatchEulerDiscreteScheduler.from_config(pipe.scheduler.config)

In [ ]:
# Reset the GPU memory tracking
torch.cuda.reset_max_memory_allocated(0)

(prompt1, prompt2), subject_token = ["A photo of a man sitting on a bench", "A photo of a man sitting on a bench with a dog"], "dog"


random.seed(0)
seeds_src = [663]
seeds_obj = [0,1,2]

for seed_src in seeds_src:
    for seed_obj in seeds_obj:
        src_image, edited_image = add_object_generated(pipe, prompt1, prompt2, subject_token, seed_src, seed_obj, show_attention=True, 
                                        extended_scale=1.05, structure_transfer_step=2, blend_steps=[15], 
                                        localization_model="attention_points_sam", display_output=True)

# Report maximum GPU memory usage in GB
max_memory_used = torch.cuda.max_memory_allocated(0) / (1024**3)  # Convert to GB
print(f"Maximum GPU memory used: {max_memory_used:.2f} GB")